In [ ]:
%run /Workspace/stream_process/setup.py

In [0]:
class batchWc():
    def __init__(self):
        self.base_data_dir = base_data_dir

    def getrawData(self):
        from pyspark.sql.functions import explode, split
        lines = (spark.read.format("text")\
            .option("lineSep", ".")\
            .load(f"{self.base_data_dir}/text_data_1.txt")
                )
        return lines.select(explode(split(lines.value, " ")).alias("word"))
    
    def getqualitydf(self, rawdf):
        from pyspark.sql.functions import trim, lower
        return ( rawdf.select(lower(trim(rawdf.word)).alias("word"))\
                    .where("word is not null")\
                    .where("word rlike '[a-z]'")
                    )
        
    def getwordcount(self, qualitydf):
        return ( qualitydf.groupBy("word").count()
                )
        
    def overwriteWord(self, resultDF):
        ( resultDF.write
                    .format("delta")
                    .mode("overwrite")
                    .saveAsTable("word_count_table")
        )

    def wordCount(self):
        print(f"\tExecuting Word Count...", end='')
        rawdf = self.getrawData()
        qualitydf = self.getqualitydf(rawdf)
        resultDF = self.getwordcount(qualitydf)
        self.overwriteWord(resultDF)
        print("Done")

## Stream Processing Auto


In [0]:
class streamWc():
    def __init__(self):
        self.base_data_dir = base_data_dir

    def getrawData(self):
        from pyspark.sql.functions import explode, split
        lines = (spark.readStream\
                .format("text")\
                .option("lineSep", ".")\
                .load(f"{self.base_data_dir}/text_data_1.txt")
                )
        return lines.select(explode(split(lines.value, " ")).alias("word"))
    
    def getqualitydf(self, rawdf):
        from pyspark.sql.functions import trim, lower
        return ( rawdf.select(lower(trim(rawdf.word)).alias("word"))\
                    .where("word is not null")\
                    .where("word rlike '[a-z]'")
                    )
        
    def getwordcount(self, qualitydf):
        return ( qualitydf.groupBy("word").count()
                )
        
    def overwriteWord(self, resultDF):
        ( resultDF.writeStream
                    .format("delta")
                    .option("checkpointLocation", f"{self.base_data_dir}/chekpoint/word_count")
                    .outputMode("complete")
                    .saveAsTable("word_count_table")
        )

    def wordCount(self):
        print(f"\tExecuting Word Count...", end='')
        rawdf = self.getrawData()
        qualitydf = self.getqualitydf(rawdf)
        resultDF = self.getwordcount(qualitydf)
        self.overwriteWord(resultDF)
        print("Done")